## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-26-17-13-50 +0000,nypost,NC mom who went missing 25 years ago arrested ...,https://nypost.com/2026/02/26/us-news/nc-mom-w...
1,2026-02-26-17-12-47 +0000,nyt,Why Are So Many Democrats Running for Californ...,https://www.nytimes.com/2026/02/26/us/californ...
2,2026-02-26-17-11-45 +0000,nypost,"IDF fatally shoots Gaza terrorist, uncovers tu...",https://nypost.com/2026/02/26/world-news/idf-f...
3,2026-02-26-17-11-00 +0000,wsj,Hillary Clinton said that she had no knowledge...,https://www.wsj.com/politics/republicans-press...
4,2026-02-26-17-10-55 +0000,nyt,He Hopes to Turn the Island of Nevis Into a Li...,https://www.nytimes.com/2026/02/26/business/st...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2539/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
55,trump,53
29,epstein,18
324,iran,16
82,new,12
84,war,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
360,2026-02-25-18-29-00 +0000,wsj,"Casey Means, President Trump’s surgeon-general...",https://www.wsj.com/politics/policy/surgeon-ge...,121
130,2026-02-26-12-32-24 +0000,nypost,"New US, Iran talks underway in Geneva as Trump...",https://nypost.com/2026/02/26/us-news/new-us-i...,104
123,2026-02-26-13-00-00 +0000,wsj,Oil lobbyists have made the case to Trump offi...,https://www.wsj.com/politics/policy/big-oil-is...,103
246,2026-02-26-00-26-00 +0000,wsj,Here’s a look at whether the math works on som...,https://www.wsj.com/politics/policy/the-math-b...,99
335,2026-02-25-20-13-37 +0000,nypost,Rivals roast ‘tired’ Rep. Tom Suozzi for dozin...,https://nypost.com/2026/02/25/us-news/rivals-r...,98


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
360,121,2026-02-25-18-29-00 +0000,wsj,"Casey Means, President Trump’s surgeon-general...",https://www.wsj.com/politics/policy/surgeon-ge...
294,66,2026-02-25-22-28-00 +0000,wsj,Spy agency blocks Congress from seeing Gabbard...,https://www.wsj.com/politics/policy/spy-agency...
130,51,2026-02-26-12-32-24 +0000,nypost,"New US, Iran talks underway in Geneva as Trump...",https://nypost.com/2026/02/26/us-news/new-us-i...
123,50,2026-02-26-13-00-00 +0000,wsj,Oil lobbyists have made the case to Trump offi...,https://www.wsj.com/politics/policy/big-oil-is...
316,49,2026-02-25-21-15-45 +0000,nypost,"JD Vance, Dr. Oz make dramatic move withholdin...",https://nypost.com/2026/02/25/us-news/jd-vance...
3,44,2026-02-26-17-11-00 +0000,wsj,Hillary Clinton said that she had no knowledge...,https://www.wsj.com/politics/republicans-press...
260,44,2026-02-25-23-48-45 +0000,nypost,Chip titan Nvidia posts record $68B in sales i...,https://nypost.com/2026/02/25/business/chip-ti...
173,42,2026-02-26-10-39-25 +0000,nypost,Kim Jong Un calls South Korea ‘most hostile en...,https://nypost.com/2026/02/26/world-news/kim-j...
246,38,2026-02-26-00-26-00 +0000,wsj,Here’s a look at whether the math works on som...,https://www.wsj.com/politics/policy/the-math-b...
200,37,2026-02-26-06-01-11 +0000,bbc,Israeli soldiers shot a Palestinian boy and st...,https://www.bbc.com/news/articles/cpqwv9vvzx9o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
